<a href="https://colab.research.google.com/gist/terris-citizen/75318a737ac7de088464cf673abafbd2/pr-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [ ]:
!pip install PyGithub pandas matplotlib ace-tools-open pytz tabulate pytest datetime timedelta

# Git Config

In [ ]:
GITHUB_TOKEN = " " ## Secret!
# Comma-delimited list
REPO_NAMES = "dev-guy/git-pr-analysis-jupyter"

# Specify How Much History to Process

In [ ]:
# 30 days of data takes about 5 minutes
max_days = 45
per_page = 50  # Max per request
max_pages = 500 # Prevent infinite loop

In [ ]:
# Calculate the cutoff date
from datetime import datetime, timedelta
import pytz

cutoff_date = datetime.now(pytz.UTC) - timedelta(days=max_days)

print (f"Processing {REPO_NAMES} for PRs after {cutoff_date}")

# Function: Compute duration between dates, ignoring weekends

Prompt: Write two python functions. The first is duration_seconds(begin, end) which takes two dates and computes the number of seconds between them and subtracts all "weekends" that overlap with the provided range. There are five global variables used by this function: timezone, begin_weekday, begin_time, end_weekday, and end_time . begin_weekday and end_weekday are short strings representing weekdays, for example Friday 2:00 PM and Monday 8:00 AM. Note that a weekend occurs in every week and correctly writing this function probably requires iterating over each week between begin and end. These global variables represent the begin and end times in each week that define the "weekend".

Write a test that calls the function multiple times and checks the return value. test at least the following cases. Add more cases that you think would be challenging.
1. the date range spans two weekends
2. the beginning of the date range falls in the middle of a weekend
3. the end of the date range falls in the middle of a weekend

In [ ]:
# Weekend definition
timezone = pytz.timezone('America/Los_Angeles')

WEEKEND_START_DAY = 4  # Friday (Mon=0, Tue=1, Wed=2, Thu=3, Fri=4, Sat=5, Sun=6)
WEEKEND_START_HOUR = 14 # 2 PM
WEEKEND_END_DAY = 0    # Monday
WEEKEND_END_HOUR = 4   # 4 AM

# The tests are based on the following:
# timezone = pytz.timezone('America/Los_Angeles')
# WEEKEND_START_DAY = 3  # Thursday (Mon=0, Tue=1, Wed=2, Thu=3, Fri=4, Sat=5, Sun=6)
# WEEKEND_START_HOUR = 18
# WEEKEND_END_DAY = 0    # Monday
# WEEKEND_END_HOUR = 8

In [ ]:
def duration_seconds(begin: datetime, end: datetime) -> int:
    """
    Calculate the number of seconds between two dates, excluding all time in
    the repeating "weekend window":
    """
    total_seconds = (end - begin).total_seconds()
    if total_seconds <= 0:
        return 0

    # 1) We'll sum the weekend overlap in `weekend_seconds`.
    weekend_seconds = 0

    # 2) Find the "Monday 00:00" of the week that includes `begin`.
    #    (Because Monday is weekday=0; we subtract `begin.weekday()` days from `begin`.)
    monday_of_current_week = datetime(
        year=begin.year,
        month=begin.month,
        day=begin.day,
        hour=0,
        minute=0,
        second=0,
        tzinfo=begin.tzinfo
    ) - timedelta(days=begin.weekday())

    # 3) We'll iterate weekly from that Monday until we've covered all weekends
    #    that might overlap [begin, end].
    current_monday = monday_of_current_week

    while True:
        # Weekend start in the current week (Thursday 18:00).
        # current_monday is Monday 00:00, so add 3 days (Thu) + set hour=18.
        w_start = current_monday + timedelta(days=WEEKEND_START_DAY)
        w_start = w_start.replace(hour=WEEKEND_START_HOUR, minute=0, second=0)

        # Weekend end in the current week (the next Monday 08:00).
        # current_monday + 7 days => next Monday 00:00, then hour=8.
        w_end = current_monday + timedelta(days=7 + WEEKEND_END_DAY)
        w_end = w_end.replace(hour=WEEKEND_END_HOUR, minute=0, second=0)

        # If the weekend starts after `end`, no more weekends to process
        if w_start >= end:
            break

        # If this entire weekend block is before `begin`, move to the next week
        if w_end <= begin:
            current_monday += timedelta(days=7)
            continue

        # Compute overlap between [w_start, w_end] and [begin, end].
        overlap_start = max(w_start, begin)
        overlap_end = min(w_end, end)

        if overlap_end > overlap_start:
            weekend_seconds += (overlap_end - overlap_start).total_seconds()

        # If this weekend end goes beyond our `end`, we're done.
        if w_end >= end:
            break

        # Otherwise, proceed to the next week's Monday
        current_monday += timedelta(days=7)

    # Subtract total weekend overlap
    net_seconds = total_seconds - weekend_seconds
    return max(int(net_seconds), 0)

In [ ]:
import pytest

# If needed, import or define the duration_seconds function here:
# from your_module import duration_seconds

def test_duration_seconds():
    # 1) Date range spans two weekends
    # Weekend is from Thursday 18:00 -> Monday 08:00.
    # Example range: Monday 2025-02-03 09:00 -> Tuesday 2025-02-18 10:00
    # This crosses these weekends:
    #   - Thu 2025-02-06 18:00 -> Mon 2025-02-10 08:00
    #   - Thu 2025-02-13 18:00 -> Mon 2025-02-17 08:00
    #
    # Total time: 15 days + 1 hour = 361 hours = 361 * 3600 = 1299600 seconds
    # Each weekend = 86 hours = 309600 seconds
    # Two weekends = 619200 seconds
    # 1299600 - 619200 = 680400
    begin = datetime(2025, 2, 3, 9, 0)
    end = datetime(2025, 2, 18, 10, 0)
    expected = 680400
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 1: expected {expected}, got {result}"

    # 2) Beginning of the date range falls in the middle of a weekend
    # Weekend: Thu 2025-02-06 18:00 -> Mon 2025-02-10 08:00
    # Example: Friday 2025-02-07 12:00 -> Monday 2025-02-10 10:00
    # From Fri 12:00 -> Mon 08:00 is entirely weekend, so no work time.
    # Only Mon 08:00 -> Mon 10:00 = 2 hours = 7200 seconds of working time.
    begin = datetime(2025, 2, 7, 12, 0)  # Friday noon
    end = datetime(2025, 2, 10, 10, 0)   # Monday 10:00 AM
    expected = 7200
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 2: expected {expected}, got {result}"

    # 3) End of the date range falls in the middle of a weekend
    # Weekend: Thu 2025-02-06 18:00 -> Mon 2025-02-10 08:00
    # Example: Monday 2025-02-03 09:00 -> Friday 2025-02-07 20:00
    # Working time is Mon 09:00 -> Thu 18:00.
    # Thu 18:00 -> Fri 20:00 is weekend, so excluded.
    #
    # Mon 09:00 -> Mon midnight = 15 hours
    # Tue full day = 24 hours
    # Wed full day = 24 hours
    # Thu 00:00 -> Thu 18:00 = 18 hours
    # Total = (15 + 24 + 24 + 18) hours = 81 hours = 291600 seconds
    begin = datetime(2025, 2, 3, 9, 0)   # Monday 9:00 AM
    end = datetime(2025, 2, 7, 20, 0)    # Friday 8:00 PM
    expected = 291600
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 3: expected {expected}, got {result}"

    # 4) No crossing of weekend
    # For example, from Tuesday 2025-02-04 10:00 -> Wednesday 2025-02-05 10:00
    # That's exactly 24 hours of working time.
    expected = 24 * 3600  # 86400
    begin = datetime(2025, 2, 4, 10, 0)
    end = datetime(2025, 2, 5, 10, 0)
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 4: expected {expected}, got {result}"

    # 5) Entirely within weekend
    # Weekend is Thu 18:00 -> Mon 08:00
    # Example: Friday 2025-02-07 10:00 -> Friday 2025-02-07 17:00
    # This is entirely during the weekend; expected = 0
    begin = datetime(2025, 2, 7, 10, 0)
    end = datetime(2025, 2, 7, 17, 0)
    expected = 0
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 5: expected {expected}, got {result}"

    # 6) End is before begin (should return 0)
    begin = datetime(2025, 2, 5, 10, 0)
    end = datetime(2025, 2, 5, 9, 0)
    expected = 0
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 6: expected {expected}, got {result}"

    # 7) Tuesday midnight to Tuesday 2 AM - should not overlap a weekend
    #    So we expect full 2 hours = 2*3600 = 7200 seconds
    begin = datetime(2025, 2, 4, 0, 0)  # Tuesday 00:00
    end = datetime(2025, 2, 4, 2, 0)   # Tuesday 02:00
    expected = 7200
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 7: expected {expected}, got {result}"

    # 8) Starts in a weekend and ends in a weekend
    # Weekend = Thursday 18:00 -> Monday 08:00.
    #
    # Example: Friday 2025-02-07 10:00 -> Sunday 2025-02-09 22:00.
    # Both begin and end are within the same weekend block:
    #   Thu (2/6) 18:00 -> Mon (2/10) 08:00
    #
    # Hence the entire interval is weekend — expect 0 seconds of working time.
    begin = datetime(2025, 2, 7, 10, 0)  # Friday 10:00
    end = datetime(2025, 2, 9, 22, 0)    # Sunday 22:00
    expected = 0
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 8: expected {expected}, got {result}"

    # 9) Begin is one hour before the weekend, end is in the weekend
    # Weekend (Thursday 18:00 -> Monday 08:00)
    # Example: Thursday (2025-02-06) 17:00 -> Friday (2025-02-07) 10:00
    # Working time is only Thu 17:00 -> Thu 18:00 = 1 hour = 3600 seconds
    begin = datetime(2025, 2, 6, 17, 0)  # Thursday 17:00
    end = datetime(2025, 2, 7, 10, 0)    # Friday 10:00
    expected = 3600  # Only 1 hour of work time
    result = duration_seconds(begin, end)
    assert result == expected, f"Case 9: expected {expected}, got {result}"

# Do not run the tests! The need specific global variables.
# test_duration_seconds()

# Function: Download PRs for One Repository

In [ ]:
processed_all_prs = True

def get_prs(repo):
    page = 0  # Start at the first page
    processed_all_for_repo = False
    repo_prs = []

    while page < max_pages:
        prs = repo.get_pulls(
            sort="created",
            direction="desc",
            state="all"
        )[page * per_page : (page + 1) * per_page]  # Manual pagination

        # Filter PRs by created_at date
        filtered_prs = [pr for pr in prs if pr.created_at >= cutoff_date]

        if not filtered_prs:
            processed_all_for_repo = True
            break  # No more PRs in the date range, exit loop

        repo_prs.extend(filtered_prs)
        page += 1  # Move to next page

    if not processed_all_for_repo:
        print(f"Loop terminated early due to max_pages ({max_pages}) failsafe")
        processed_all_prs = False

    return repo_prs

# Function: Process PR Reviews for One Repository

In [ ]:
pr_data = []

# Reviewers is a map from user name to { repo/PR/date }
reviewers = {}

now = datetime.now(pytz.utc)

def get_pr_data(repo, prs):
    for pr in prs:
        if pr.user.login == "dependabot[bot]":
            continue

        first_review_at = None
        ready_for_review_at = None
        time_to_merge = None
        time_to_first_review = None
        open_duration = None
        reviews = None

        if pr.draft:
            if pr.closed_at is not None:
                continue
        else:
            # When was the PR ready for review?
            issue = repo.get_issue(pr.number)
            timeline = issue.get_timeline()
            ready_for_review_at = pr.created_at

            for event in timeline:
                if event.event == "ready_for_review":
                    ready_for_review_at = event.created_at
                    break

            reviews = pr.get_reviews()
            # remove reviews by the author and CodeRabbit
            reviews = [
                review for review in reviews
                if review.user.login != pr.user.login and
                    review.user.login != "coderabbitai[bot]"
            ]

            if len(reviews) == 0:
                if pr.closed_at is not None:
                    continue
            else:
                # reviewed_by is a map of user name and review
                reviewed_by = {}
                for review in reviews:
                    if review.user.login not in reviewed_by:
                        reviewed_by[review.user.login] = review

                # For each item in submitted_by, add the item to reviewers
                for user, review in reviewed_by.items():
                    if user not in reviewers:
                        reviewers[user] = []
                    reviewers[user].append({
                        "Repository": pr.base.repo.full_name,
                        "PR": pr.number,
                        "Submitted At": review.submitted_at,
                        "Submitted By": user,
                    })

                first_review_at = reviews[0].submitted_at
                if first_review_at < ready_for_review_at:
                    # First review occurred before marked as not a draft
                    time_to_first_review = duration_seconds(pr.created_at, first_review_at) / 3600
                else:
                    time_to_first_review = duration_seconds(ready_for_review_at, first_review_at) / 3600

            if pr.closed_at is None:
                open_duration = duration_seconds(ready_for_review_at, now) / 3600

            if pr.merged_at is not None:
                time_to_merge = duration_seconds(pr.created_at, pr.merged_at) / 3600

        pr_data.append({
            "Repository": pr.base.repo.full_name,
            "PR": pr.number,
            "Author": pr.user.login,
            "Created At": pr.created_at,
            "Ready For Review At": ready_for_review_at,
            "First Review At": first_review_at,
            "Time to First Review (hrs)": time_to_first_review,
            "Merged At": pr.merged_at,
            "Time to Merge (hrs)": time_to_merge,
            "Number of Reviews": len(reviews) if reviews is not None else None,
            "Waiting for First Review (hrs)": open_duration if reviews is not None and len(reviews) == 0 else None,
            "Is Draft": pr.draft,
            "Closed At": pr.closed_at,
            "Open (hrs)": open_duration,
        })

# Download PRs

In [ ]:
from github import Github
g = Github(GITHUB_TOKEN)

In [ ]:
for repo_name in REPO_NAMES.split(','):
    repo_name2 = repo_name.strip()
    repo = g.get_repo(repo_name2)
    print (f"Collecting PRs for {repo_name2}")
    prs = get_prs(repo)
    print (f"Processing PRs for {repo_name2}")
    get_pr_data(repo, prs)

# Begin Analysis

In [ ]:
if len(pr_data) == 0:
    raise Exception("No PRs found")

In [ ]:
import pandas as pd

# Basic statistics

min_created_at = min(pr["Created At"] for pr in pr_data)
max_created_at = max(pr["Created At"] for pr in pr_data)
min_merged_at = min((pr["Merged At"] for pr in pr_data if pr["Merged At"] is not None), default=None)
max_merged_at = max((pr["Merged At"] for pr in pr_data if pr["Merged At"] is not None), default=None)

num_prs = len(pr_data)
num_drafts = sum(1 for pr in pr_data if pr["Is Draft"])
num_open = sum(1 for pr in pr_data if pr["Open (hrs)"] is not None)
num_waiting = sum(1 for pr in pr_data if pr["Waiting for First Review (hrs)"] is not None)

# Avg number of reviews per PR that has reviews
valid_reviews = [
    pr["Number of Reviews"]
    for pr in pr_data
    if pr["Number of Reviews"] is not None and pr["Number of Reviews"] > 0
]

if valid_reviews:
    avg_num_reviews = sum(valid_reviews) / len(valid_reviews)
else:
    avg_num_reviews = 0

summary_df = pd.DataFrame({
    "Metric": [
        "Cutoff Date",
        "Processed All PRs",
        "PRs",
        "Drafts",
        "Open",
        "Waiting for First Review",
        "Avg Reviews per PR",
        "Min Created At",
        "Max Created At",
        "Min Merged At",
        "Max Merged At",
    ],
    "Value": [
        cutoff_date,
        processed_all_prs,
        num_prs,
        num_drafts,
        num_open,
        num_waiting,
        avg_num_reviews,
        min_created_at,
        max_created_at,
        min_merged_at,
        max_merged_at,
    ]})

# Display the summary table
summary_df

In [ ]:
# Raw PR data
import ace_tools_open as tools

df = pd.DataFrame(pr_data)
tools.display_dataframe_to_user(name="PRs", dataframe=df)

In [ ]:
# Reviews by Week
reviewer_data = []

for user, reviews in reviewers.items():
    # Group reviews by week
    week_counts = {}

    for review in reviews:
        # Parse the submission date
        # submitted_at = datetime.fromisoformat(review["Submitted At"].replace("Z", "+00:00"))
        submitted_at = review["Submitted At"]

        # Calculate the beginning of the week (Monday)
        week_begin = submitted_at - timedelta(days=submitted_at.weekday())
        week_begin = week_begin.replace(hour=0, minute=0, second=0, microsecond=0)

        # Calculate the end of the week (Sunday)
        week_end = week_begin + timedelta(days=6, hours=23, minutes=59, seconds=59)

        # Format dates for the key
        week_key = (week_begin.date(), week_end.date())

        # Count reviews per week
        if week_key in week_counts:
            week_counts[week_key] += 1
        else:
            week_counts[week_key] = 1

    # Add data to our table
    for (week_begin, week_end), count in week_counts.items():
        reviewer_data.append({
            "Week": 0,
            "Week Begin": week_begin,
            "Week End": week_end,
            "Reviewer": user,
            "PRs": count,
        })

df_reviewers = pd.DataFrame(reviewer_data)

# Create a mapping of unique weeks to week numbers (0, -1, -2, etc.)
unique_weeks = df_reviewers["Week Begin"].unique()
week_to_number = {week: -i for i, week in enumerate(unique_weeks)}

# Add the week number column
df_reviewers["Week"] = df_reviewers["Week Begin"].map(week_to_number)

# Sort by user and week begin date
df_reviewers = df_reviewers.sort_values(by=["Week", "PRs", "Reviewer"],ascending=[False, False, True])

tools.display_dataframe_to_user(name="Reviews By Week", dataframe=df_reviewers)

In [ ]:
# Time metrics
def analyze_time_ranges(series, metric_name):
    total = len(series.dropna())

    # Calculate counts and percentages for each range
    ranges = {
        "< 1 day": (series <= 24).sum(),
        "1-2 days": ((series > 24) & (series <= 48)).sum(),
        "2-3 days": ((series > 48) & (series <= 72)).sum(),
        "3-4 days": ((series > 72) & (series <= 96)).sum(),
        "> 4 days": (series > 96).sum()
    }

    # Convert to percentages
    percentages = {k: (v/total * 100) for k, v in ranges.items()}

    print(f"\n{metric_name} Analysis:")
    print("-" * 50)
    for range_name, count in ranges.items():
        print(f"{range_name}: {count} PRs ({percentages[range_name]:.1f}%)")

In [ ]:
# Time metrics
analyze_time_ranges(df["Open (hrs)"], "Open Time")
analyze_time_ranges(df["Waiting for First Review (hrs)"], "Waiting for First Review")
analyze_time_ranges(df["Time to First Review (hrs)"], "Time to First Review")
analyze_time_ranges(df["Time to Merge (hrs)"], "Time to Merge")

In [ ]:
# Time to first review and merge distribution charts
import matplotlib.pyplot as plt

def plot_time_ranges(df, metric_col, title):
    total = len(df[metric_col].dropna())

    ranges = {
        "< 1 day": (df[metric_col] <= 24).sum() / total * 100,
        "1-2 days": ((df[metric_col] > 24) & (df[metric_col] <= 48)).sum() / total * 100,
        "2-3 days": ((df[metric_col] > 48) & (df[metric_col] <= 72)).sum() / total * 100,
        "3-4 days": ((df[metric_col] > 72) & (df[metric_col] <= 96)).sum() / total * 100,
        "> 4 days": (df[metric_col] > 96).sum() / total * 100
    }

    plt.figure(figsize=(10, 6))
    plt.bar(ranges.keys(), ranges.values())
    plt.title(title)
    plt.ylabel("Percentage of PRs")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot time metrics
plot_time_ranges(df, "Open (hrs)", "Open Time Distribution")
plot_time_ranges(df, "Waiting for First Review (hrs)", "Waiting for First Review Time Distribution")
plot_time_ranges(df, "Time to First Review (hrs)", "Time to First Review Distribution")
plot_time_ranges(df, "Time to Merge (hrs)", "Time to Merge Distribution")

In [ ]:
# Metrics by author
author_metrics = df.groupby('Author').agg(
    Total=('PR', 'count'),
    Merged=('Merged At', lambda x: x.notna().sum()),
    Drafts=('Is Draft', 'sum'),
    Open=('Open (hrs)', lambda x: x.notna().sum()),
    Waiting_For_Review=('Number of Reviews', lambda x: (x == 0).sum()),
    Max_Waiting_For_First_Review_Hrs=('Waiting for First Review (hrs)', 'max'),
    Median_Open_Hrs=('Open (hrs)', 'median')
).sort_values('Total', ascending=False)

author_metrics

In [ ]:
# Calculate metrics for time to first review by author
first_review_stats = df.groupby('Author').agg({
    'Time to First Review (hrs)': ['mean', 'median', 'max']
}).sort_values(('Time to First Review (hrs)', 'max'), ascending=False)

# Calculate metrics for waiting for first review time by author
waiting_for_review_stats = df.groupby('Author').agg({
    'Waiting for First Review (hrs)': ['mean', 'median', 'max']
}).sort_values(('Waiting for First Review (hrs)', 'max'), ascending=False)

# Calculate metrics for open time by author
open_stats = df.groupby('Author').agg({
    'Open (hrs)': ['mean', 'median', 'max']
}).sort_values(('Open (hrs)', 'max'), ascending=False)

# Calculate metrics for time to merge by author
merge_stats = df.groupby('Author').agg({
    'Time to Merge (hrs)': ['mean', 'median', 'max']
}).sort_values(('Time to Merge (hrs)', 'max'), ascending=False)

# Format the tables
first_review_stats.columns = ['Avg Time to First Review (hrs)', 'Median Time to First Review (hrs)', 'Max Time to First Review (hrs)']
waiting_for_review_stats.columns = ['Avg Waiting for Review (hrs)', 'Median Waiting for Review (hrs)', 'Max Waiting for Review (hrs)']
merge_stats.columns = ['Avg Time to Merge (hrs)', 'Median Time to Merge (hrs)', 'Max Time to Merge (hrs)']
open_stats.columns = ['Avg Open (hrs)', 'Median Open (hrs)', 'Max Open (hrs)']

print(first_review_stats.round(2))
print(waiting_for_review_stats.round(2))
print(merge_stats.round(2))
print(open_stats.round(2))